In [1]:
from datetime import datetime
import logging
import sys
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
import optuna
from optuna.integration import XGBoostPruningCallback

import warnings
warnings.filterwarnings('ignore')

optuna_dir = "..//optuna"
folder_dir = "//xgb"

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')

train_df = pd.concat([train_df, train_extra_df], ignore_index=True)

train_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity', 'price'
]
test_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity',
]

cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert all to be the same type
train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

target = "price"
X = train_df.drop(columns=[target])
y = train_df[target]

# Only keeping important columns
model_features = [
    'weight_capacity', 'color', 'compartments', 'brand', 'material', 'is_waterproof'
]
X = X[model_features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
def objective(trial):
    param = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        # "grow_policy", trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        # "colsample_bytree": trial.suggest_float("colsample_bytree", 0.25, 0.35),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),

        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.05),
        "max_depth": trial.suggest_int("max_depth", 3, 14),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 1),
        # "subsample": trial.suggest_float("subsample", 0.2, 0.25),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.5, 2.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-6, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),

        "seed": 42,
        "verbosity": 0,
    }
    
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dvalid = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)
    pruning_callback = XGBoostPruningCallback(trial, observation_key="validation_0-rmse")    
    bst = xgb.train(
        params=param,
        dtrain=dtrain,
        num_boost_round=700,
        evals=[(dtrain, "train"), (dvalid, "validation_0")],
        early_stopping_rounds=50,
        verbose_eval=False,
        callbacks=[pruning_callback]
    )
    y_pred = bst.predict(dvalid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    return rmse

# Set up logging for Optuna
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

# Create a study (using a SQLite database for persistence)
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///{os.path.join(optuna_dir, folder_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction="minimize")

# Run the optimization for 50 trials (adjust as needed)
study.optimize(objective, n_trials=50)

[I 2025-02-05 20:46:02,205] A new study created in RDB with name: 2025-02-05_20-46


A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46
A new study created in RDB with name: 2025-02-05_20-46


[I 2025-02-05 20:46:58,119] Trial 0 finished with value: 38.874651392883635 and parameters: {'colsample_bytree': 0.30368609920096834, 'learning_rate': 0.020787806406362644, 'min_child_weight': 7, 'gamma': 0.3923739691441668, 'subsample': 0.23088715928463702, 'reg_alpha': 0.8667906624916685, 'reg_lambda': 1.902405231538214e-06}. Best is trial 0 with value: 38.874651392883635.


Trial 0 finished with value: 38.874651392883635 and parameters: {'colsample_bytree': 0.30368609920096834, 'learning_rate': 0.020787806406362644, 'min_child_weight': 7, 'gamma': 0.3923739691441668, 'subsample': 0.23088715928463702, 'reg_alpha': 0.8667906624916685, 'reg_lambda': 1.902405231538214e-06}. Best is trial 0 with value: 38.874651392883635.
Trial 0 finished with value: 38.874651392883635 and parameters: {'colsample_bytree': 0.30368609920096834, 'learning_rate': 0.020787806406362644, 'min_child_weight': 7, 'gamma': 0.3923739691441668, 'subsample': 0.23088715928463702, 'reg_alpha': 0.8667906624916685, 'reg_lambda': 1.902405231538214e-06}. Best is trial 0 with value: 38.874651392883635.
Trial 0 finished with value: 38.874651392883635 and parameters: {'colsample_bytree': 0.30368609920096834, 'learning_rate': 0.020787806406362644, 'min_child_weight': 7, 'gamma': 0.3923739691441668, 'subsample': 0.23088715928463702, 'reg_alpha': 0.8667906624916685, 'reg_lambda': 1.902405231538214e-06}

[I 2025-02-05 20:47:56,411] Trial 1 finished with value: 38.87586999206969 and parameters: {'colsample_bytree': 0.3161726799343499, 'learning_rate': 0.014105748310604781, 'min_child_weight': 9, 'gamma': 0.00658619391068338, 'subsample': 0.20331668662152036, 'reg_alpha': 1.0059838814075577, 'reg_lambda': 1.0335420916485014e-06}. Best is trial 0 with value: 38.874651392883635.


...

Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.
Trial 47 pruned. Trial was pruned at iteration 0.


[I 2025-02-05 21:01:00,642] Trial 48 pruned. Trial was pruned at iteration 0.


Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.


[I 2025-02-05 21:01:01,273] Trial 49 pruned. Trial was pruned at iteration 0.


Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
